In [1]:
import csv
import numpy as np
import pandas as pd

import cv2

import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
# remove prefixes
import os
path = '/media/stanislau/82db778e-0496-450c-9b25-d1e50a90e476/data/data4stas/01_data_cls'
os.chdir(path)

In [ ]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [137]:
def process_file(path):
    df = pd.read_csv(path)
    
    x = []
    y = []

    for index, row in df.iterrows():
        x.append(cv2.imread(row['FilePath'], cv2.IMREAD_GRAYSCALE))
        y.append(float(row['ClassId']))

    x = np.array(x)
    y = np.array(y)
    
    num_classes = len(np.unique(y))

    x = x.reshape(x.shape + (-1,))
    y = y - 1
    y = keras.utils.to_categorical(y, num_classes=num_classes)

    return x, y, num_classes

def process_file2(path, classes):
    df = pd.read_csv(path)
    
    x = []
    y = []

    for index, row in df.iterrows():
        x.append(cv2.imread(row['FilePath'], cv2.IMREAD_GRAYSCALE))
        y.append(float(row['ClassId']))

    x = np.array(x)
    y = np.array(y)
    
    x = x.reshape(x.shape + (-1,))
    y = y - 1
    
    indexes = np.argwhere(y < classes)
    
    y = y[indexes]
    x = x[indexes.ravel()]
    
    y = keras.utils.to_categorical(y, num_classes=classes)

    return x, y, classes


def process_line(line):
    pass

def generate_arrays_from_file(path):
    df = pd.read_csv(path)
    
    while 1:
        f = open(path)
        for line in f:
            # create numpy arrays of input data
            # and labels, from each line in the file
            x1, x2, y = process_line(line)
            yield ({'input_1': x1, 'input_2': x2}, {'output': y})
        f.close()

In [115]:
x, y, num_classes = process_file('Alexander_cls_train.csv')

In [160]:
x, y, num_classes = process_file2('Alexander_cls_train.csv', classes=classes)

(50, 200, 200, 1)

In [182]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=7,
    width_shift_range=0.4,
    height_shift_range=0.4,
#     shear_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True,
#     zoom_range=0.2,
    fill_mode='reflect')

In [183]:
datagen.fit(x)

In [ ]:
%pylab inline

In [71]:
filters = [
    (2, 2),
    (3, 3),
    (5, 5),
    (7, 7),
    (10, 10),
    (12, 12)
]

filter_size = filters[1]

In [189]:
model = Sequential()
# input: 200x200 grayscale images -> (200, 200, 1) tensors.
# this applies 8 convolution filters of size 3x3 each.
model.add(Conv2D(64, filter_size, kernel_regularizer=regularizers.l2(0.01), activation='relu', input_shape=(200, 200, 1)))
model.add(Conv2D(64, filter_size, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(Conv2D(32, (2, 2), activation='relu'))
# model.add(Conv2D(16, (2, 2), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(Conv2D(32, (2, 2), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Conv2D(128, filter_size, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(Conv2D(128, filter_size, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(Conv2D(64, (2, 2), activation='relu'))
# model.add(Conv2D(32, (2, 2), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Conv2D(256, filter_size, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(Conv2D(256, filter_size, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(Conv2D(256, filter_size, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(Conv2D(128, (2, 2), activation='relu'))
# model.add(Conv2D(64, (2, 2), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Dropout(0.25))

model.add(Conv2D(512, filter_size, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(Conv2D(512, filter_size, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(Conv2D(512, filter_size, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(Conv2D(512, (2, 2), activation='relu'))
# model.add(Conv2D(128, (2, 2), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(512, filter_size, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(Conv2D(512, filter_size, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(Conv2D(512, filter_size, kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# # # model.add(Conv2D(512, (2, 2), activation='relu'))
# # # model.add(Conv2D(128, (2, 2), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (2, 2), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(128, (2, 2), kernel_regularizer=regularizers.l2(0.01), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(512, (2, 2), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(num_classes, kernel_regularizer=regularizers.l2(0.01), activation='softmax'), )
model.add(Dropout(0.5))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [ ]:
# model.fit(x, y, batch_size=8, epochs=10, validation_data=(x_train, y_train))
model.fit(x, y, batch_size=32, epochs=10)

In [191]:
batch_size = 50
model.fit_generator(datagen.flow(x, y, batch_size=batch_size),
                    steps_per_epoch=len(x) / batch_size * 10, epochs=10)

Epoch 1/10
10/10 [==============================] - 2s - loss: nan - acc: 0.1000     
Epoch 2/10
10/10 [==============================] - 1s - loss: nan - acc: 0.1000     
Epoch 3/10
 9/10 [==========================>...] - ETA: 0s - loss: nan - acc: 0.1000

KeyboardInterrupt: 

In [37]:
batch_size = 32
model.fit_generator(datagen.flow(x, y, batch_size=batch_size),
                    steps_per_epoch=len(x) / batch_size * 2, epochs=700, use_multiprocessing=True, workers=20)

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:1987: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/700
35/34 [==============================] - 20s - loss: 25.5768 - acc: 0.0107    
Epoch 2/700
35/34 [==============================] - 19s - loss: 23.0350 - acc: 0.0098    
Epoch 3/700
35/34 [==============================] - 20s - loss: 21.6410 - acc: 0.0080    
Epoch 4/700
35/34 [==============================] - 21s - loss: 19.8045 - acc: 0.0134    
Epoch 5/700
35/34 [==============================] - 20s - loss: 18.7373 - acc: 0.0134    
Epoch 6/700
35/34 [==============================] - 21s - loss: 17.3927 - acc: 0.0152    
Epoch 7/700
35/34 [==============================] - 20s - loss: 16.1347 - acc: 0.0214    
Epoch 8/700
35/34 [==============================] - 20s - loss: 15.2032 - acc: 0.0250    
Epoch 9/700
35/34 [==============================] - 20s - loss: 14.5017 - acc: 0.0304    
Epoch 10/700
35/34 [==============================] - 18s - loss: 13.9926 - acc: 0.0325    
Epoch 11/700
35/34 [==============================] - 16s - loss: 13.8043 - acc: 0.0320  

35/34 [==============================] - 20s - loss: 9.4127 - acc: 0.2679    
Epoch 91/700
35/34 [==============================] - 20s - loss: 9.1974 - acc: 0.2795    
Epoch 92/700
35/34 [==============================] - 18s - loss: 9.3202 - acc: 0.2555    
Epoch 93/700
35/34 [==============================] - 15s - loss: 9.5363 - acc: 0.2136    
Epoch 94/700
35/34 [==============================] - 20s - loss: 9.3087 - acc: 0.2696    
Epoch 95/700
35/34 [==============================] - 20s - loss: 9.3063 - acc: 0.2777    
Epoch 96/700
35/34 [==============================] - 20s - loss: 9.2484 - acc: 0.2884    
Epoch 97/700
35/34 [==============================] - 20s - loss: 9.0494 - acc: 0.2857    
Epoch 98/700
35/34 [==============================] - 20s - loss: 9.0541 - acc: 0.2893    
Epoch 99/700
35/34 [==============================] - 20s - loss: 9.0733 - acc: 0.2875    
Epoch 100/700
35/34 [==============================] - 20s - loss: 9.1548 - acc: 0.2991    
Epoch 101/7

35/34 [==============================] - 19s - loss: 9.1009 - acc: 0.3330    
Epoch 180/700
35/34 [==============================] - 20s - loss: 9.5463 - acc: 0.3179    
Epoch 181/700
35/34 [==============================] - 20s - loss: 9.0215 - acc: 0.3214    
Epoch 182/700
35/34 [==============================] - 20s - loss: 9.1607 - acc: 0.3366    
Epoch 183/700
35/34 [==============================] - 19s - loss: 9.0390 - acc: 0.3259    
Epoch 184/700
35/34 [==============================] - 19s - loss: 8.7141 - acc: 0.3413    
Epoch 185/700
35/34 [==============================] - 17s - loss: 9.5744 - acc: 0.2806    
Epoch 186/700
35/34 [==============================] - 16s - loss: 8.8319 - acc: 0.2952    
Epoch 187/700
35/34 [==============================] - 20s - loss: 9.5165 - acc: 0.2964    
Epoch 188/700
35/34 [==============================] - 20s - loss: 9.0905 - acc: 0.3214    
Epoch 189/700
35/34 [==============================] - 20s - loss: 9.1260 - acc: 0.3384    
Ep

35/34 [==============================] - 15s - loss: 10.0935 - acc: 0.2084    
Epoch 269/700
35/34 [==============================] - 19s - loss: 9.0007 - acc: 0.3018    
Epoch 270/700
35/34 [==============================] - 20s - loss: 9.0860 - acc: 0.3286    
Epoch 271/700
35/34 [==============================] - 19s - loss: 8.9397 - acc: 0.3482    
Epoch 272/700
35/34 [==============================] - 19s - loss: 9.1122 - acc: 0.3491    
Epoch 273/700
35/34 [==============================] - 19s - loss: 9.2856 - acc: 0.3321    
Epoch 274/700
35/34 [==============================] - 19s - loss: 9.2005 - acc: 0.3491    
Epoch 275/700
35/34 [==============================] - 19s - loss: 8.8905 - acc: 0.3625    
Epoch 276/700
35/34 [==============================] - 20s - loss: 9.1143 - acc: 0.3473    
Epoch 277/700
35/34 [==============================] - 18s - loss: 9.5159 - acc: 0.3119    
Epoch 278/700
35/34 [==============================] - 15s - loss: 9.3428 - acc: 0.3084    
E

35/34 [==============================] - 20s - loss: 8.7719 - acc: 0.3696    
Epoch 358/700
35/34 [==============================] - 19s - loss: 9.2297 - acc: 0.3313    
Epoch 359/700
35/34 [==============================] - 18s - loss: 8.9131 - acc: 0.3629    
Epoch 360/700
35/34 [==============================] - 16s - loss: 9.1334 - acc: 0.3422    
Epoch 361/700
35/34 [==============================] - 17s - loss: 9.1851 - acc: 0.3050    
Epoch 362/700
35/34 [==============================] - 19s - loss: 9.2092 - acc: 0.3500    
Epoch 363/700
35/34 [==============================] - 20s - loss: 9.2775 - acc: 0.3589    
Epoch 364/700
35/34 [==============================] - 20s - loss: 8.7088 - acc: 0.3830    
Epoch 365/700
35/34 [==============================] - 19s - loss: 9.0057 - acc: 0.3714    
Epoch 366/700
35/34 [==============================] - 19s - loss: 9.1425 - acc: 0.3455    
Epoch 367/700
35/34 [==============================] - 19s - loss: 9.0842 - acc: 0.3670    
Ep

35/34 [==============================] - 19s - loss: 9.2841 - acc: 0.3295    
Epoch 447/700
35/34 [==============================] - 19s - loss: 8.8789 - acc: 0.3795    
Epoch 448/700
35/34 [==============================] - 19s - loss: 9.1325 - acc: 0.3464    
Epoch 449/700
35/34 [==============================] - 19s - loss: 8.9753 - acc: 0.3661    
Epoch 450/700
35/34 [==============================] - 19s - loss: 9.1936 - acc: 0.3402    
Epoch 451/700
35/34 [==============================] - 20s - loss: 8.9170 - acc: 0.3795    
Epoch 452/700
35/34 [==============================] - 18s - loss: 9.6560 - acc: 0.2812    
Epoch 453/700
35/34 [==============================] - 15s - loss: 9.3996 - acc: 0.2956    
Epoch 454/700
35/34 [==============================] - 19s - loss: 8.9762 - acc: 0.3473    
Epoch 455/700
35/34 [==============================] - 19s - loss: 8.7468 - acc: 0.3688    
Epoch 456/700
35/34 [==============================] - 19s - loss: 9.2547 - acc: 0.3696    
Ep

Process Process-87:
Process Process-95:
Process Process-94:
Process Process-83:
Traceback (most recent call last):
Process Process-96:
Process Process-81:
Process Process-93:
Process Process-92:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-86:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Process Process-82:
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/pr

KeyboardInterrupt: 

In [40]:
print(model.evaluate_generator(test_datagen.flow(x, y, batch_size=32), steps=len(x) / 32))

[11.531507627813665, 0.31531531547640895]


In [38]:
test_datagen = ImageDataGenerator(
    featurewise_center=True,
#     featurewise_std_normalization=True
)

test_datagen.fit(x)

test_paths = ['Alexander_cls_train.csv',
             'Alexander_cls_test.csv',
             'Alexander_cls_test1.csv',
             'Alexander_cls_test2.csv',
             'Alexander_cls_test3.csv',
             'Alexander_cls_test4.csv']

for test_path in test_paths:
    x_test, y_test, num_classes = process_file(test_path)
#     test_datagen.fit(x_test)
    score = model.evaluate_generator(test_datagen.flow(x_test, y_test, batch_size=32), steps=len(x_test) / 32)
    print(test_path + ' : ' + ', '.join(str(x) for x in score))

Alexander_cls_train.csv : 11.5315075385, 0.315315315369
Alexander_cls_test.csv : 13.6336499102, 0.181531531545
Alexander_cls_test1.csv : 11.6636511485, 0.306306306414
Alexander_cls_test2.csv : 14.4012747825, 0.131531531746
Alexander_cls_test3.csv : 13.5366840569, 0.187387387495
Alexander_cls_test4.csv : 14.9329902632, 0.100900900955


In [ ]:
model.evaluate(x, y, batch_size=32)

In [ ]:
y_predict = model.predict(x_train, batch_size=32)

In [ ]:
np.unique(np.argmax(y_predict, axis=1))

In [172]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_133 (Conv2D)          (None, 198, 198, 64)      640       
_________________________________________________________________
max_pooling2d_81 (MaxPooling (None, 99, 99, 64)        0         
_________________________________________________________________
conv2d_134 (Conv2D)          (None, 97, 97, 128)       73856     
_________________________________________________________________
max_pooling2d_82 (MaxPooling (None, 48, 48, 128)       0         
_________________________________________________________________
conv2d_135 (Conv2D)          (None, 46, 46, 256)       295168    
_________________________________________________________________
max_pooling2d_83 (MaxPooling (None, 23, 23, 256)       0         
_________________________________________________________________
conv2d_136 (Conv2D)          (None, 21, 21, 512)       1180160   
__________

In [ ]:
# спросить про 500 samples на класс (хорошо, но можно оверфитнуться)
# когда 50 сэмплов хорошо, обычно плохо